## Solution (b) - Perform data discretization using Chi-Merge method

In [34]:
## crucial libs
import pandas as pd
from collections import Counter
import numpy as np


In [35]:
#now reading Iris dataset from the url
iris = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)

# as the data set does not have column header, adding header
iris.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'iris_type']

#Lets see some data
print(iris.shape)
print(iris.head())
print(iris.info())

(150, 5)
   sepal_length  sepal_width  petal_length  petal_width    iris_type
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   iris_type     150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 5.3+ KB
None


### Chi-Merge Algorithm implementaion

In [36]:
## now implementing chi-merge algorithm
def chi_merge(data, attr, label, max_intervals):
    
    # if division by zero error, ignore
    np.seterr(divide='ignore', invalid='ignore')
    
    # distinct sorted values of the given features
    distinct_vals = sorted(set(data[attr]))  
    
    # distinct sorted labels
    labels = sorted(set(data[label]))  
    
    # dictionary of counts for each label
    empty_count = {l: 0 for l in labels}  
    
    # initializing the intervals for the given attribute; to start with each row is taken as an interval
    intervals = [[distinct_vals[i], distinct_vals[i]] for i in range(len(distinct_vals))] 
 
    # Keep applying chimerge process as long as we reach the max_intervals condition
    while len(intervals) > max_intervals:
        
        #Array to hold the chi values for this iteration
        chi = []
        
        #Calculate chi values for each consecutive intervals in this iteration
        for i in range(len(intervals)-1):
            
            # Indexes of the attribute that falls between given interval 
            obs0 = data[data[attr].between(intervals[i][0], intervals[i][1])]
            obs1 = data[data[attr].between(intervals[i+1][0], intervals[i+1][1])]
            total = len(obs0) + len(obs1)
            
            #Count the values for each label for given attribute
            count_0 = np.array([v for i, v in {**empty_count, **Counter(obs0[label])}.items()])
            count_1 = np.array([v for i, v in {**empty_count, **Counter(obs1[label])}.items()])
            count_total = count_0 + count_1
            
            #Caclculate expected values
            expected_0 = count_total*sum(count_0)/total
            expected_1 = count_total*sum(count_1)/total
  
            # Calculate the Chi2 value
            chi_ = (count_0 - expected_0)**2/expected_0 + (count_1 - expected_1)**2/expected_1
            chi_ = np.nan_to_num(chi_) # Deal with the zero counts
            
            # Finally do the summation for Chi2 and append it to list of chi values
            chi.append(sum(chi_)) 
            
        
        #Find the minimum chi for the current iteration
        min_chi = min(chi)  
 
        #Find the first index with minumum chi
        for i, v in enumerate(chi):
            if v == min_chi:
                min_chi_index = i # Find the index of the interval to be merged
                break
                
        
        # Prepare for the merged array
        new_intervals = [] 
        skip = False
        done = False
        
        #Merge the intervals found at min_chi_index with next interval
        for i in range(len(intervals)):
            if skip:
                skip = False
                continue
            if i == min_chi_index and not done: # Merge the intervals
                t = intervals[i] + intervals[i+1]
                new_intervals.append([min(t), max(t)])
                skip = True
                done = True
            else:
                new_intervals.append(intervals[i])
        
        #Start the chimerge with new set of merged intervals
        intervals = new_intervals
    
    # split points for the attribute
    print('\nSplit points:',attr)
    for i in intervals:
        print(i[0])
        
    # intervals for the attribute
    print('\nIntervals:', attr)
    for i in intervals:
        print('[', i[0], ',', i[1], ']', sep='')
        

In [37]:
# applying chi-merge algo on each feature with stopping criteria of maximum 6 intervals
print('Result:')
for attr in ['sepal_length','sepal_width', 'petal_length', 'petal_width']:
    chi_merge(data=iris, attr=attr, label='iris_type', max_intervals=6)

Result:

Split points: sepal_length
4.3
4.9
5.0
5.5
5.8
7.1

Intervals: sepal_length
[4.3,4.8]
[4.9,4.9]
[5.0,5.4]
[5.5,5.7]
[5.8,7.0]
[7.1,7.9]

Split points: sepal_width
2.0
2.3
2.5
2.9
3.0
3.4

Intervals: sepal_width
[2.0,2.2]
[2.3,2.4]
[2.5,2.8]
[2.9,2.9]
[3.0,3.3]
[3.4,4.4]

Split points: petal_length
1.0
3.0
4.5
4.8
5.0
5.2

Intervals: petal_length
[1.0,1.9]
[3.0,4.4]
[4.5,4.7]
[4.8,4.9]
[5.0,5.1]
[5.2,6.9]

Split points: petal_width
0.1
1.0
1.4
1.7
1.8
1.9

Intervals: petal_width
[0.1,0.6]
[1.0,1.3]
[1.4,1.6]
[1.7,1.7]
[1.8,1.8]
[1.9,2.5]
